In [1]:
import pandas as pd
import numpy as np

import bokeh.plotting
import bokeh.io
import bokeh.application
import bokeh.application.handlers
import bokeh.layouts
import bokeh.models
import bokeh.palettes

from bokeh.tile_providers import CARTODBPOSITRON, get_provider

tile_provider = get_provider(CARTODBPOSITRON)
bokeh.io.output_notebook()

Loading BokehJS ...

# Import the data

In [2]:
# Load in the metadata
ferm_locations = pd.read_csv("./data/ferm.csv")

# Split the Lat-Long column into two columns, Lat and Long
ferm_locations[['Lat', 'Long']] = ferm_locations['Lat-Long'].str.split(', ', 1, expand=True)

# Create a new dataframe only if the point has a valid Lat and Long
ferm_locations_final = ferm_locations[ferm_locations['Lat'].notna()].copy()

# Convert from a string, and put in columns called Latitude and Longitude
ferm_locations_final.loc[:,'Latitude1'] = pd.to_numeric(ferm_locations_final['Lat'],errors='coerce')
ferm_locations_final.loc[:,'Longitude1'] = pd.to_numeric(ferm_locations_final['Long'],errors='coerce')

# Clean it

In [3]:
ferm_locations_final.Class.unique()

array(['Dairy', 'Control', 'Bread', 'Bean', 'Drink', 'Vegetable', 'Fruit',
       'Fish', 'Vinegar', 'Rice', 'Shellfish', 'Intestine', 'Meat',
       'Crustacean', 'Unclear'], dtype=object)

In [4]:
def mercator_creator(df, lat='lat', lon='lon'):
    # from https://www.youtube.com/watch?v=BojxegBh9_4
    
    k = 6378137
    df['x'] = k * np.radians(df[lon])
    df['y'] = np.log(np.tan((90 + df[lat]) * np.pi / 360)) * k
    
    return df

In [5]:
ferm_locations_final = mercator_creator(ferm_locations_final, lat='Latitude1', lon='Longitude1')

# Make the selector and cds

In [6]:
ferm_locations_final

,Sample_ID,Type,Location_made,Grouping,Class,Acquisition_date,Ferment_days,Extract_date,Location_Type_made,Country_made,...,Arrived_frozen,DNA_Status,Arrival_date,Lat-Long,Lat,Long,Latitude1,Longitude1,x,y
0,LNM1,Fresh_Cheese,Fontrieu,NaN,Dairy,"December 5, 2021",NaN,"December 6, 2021",Farm,France,...,Yes,Switzerland,"January 13, 2022","43.66547031455824, 2.45386246303461",43.66547031455824,2.45386246303461,43.665470,2.453862,2.731627e+05,5.413818e+06
1,LNM2,Roquefort_Carles,Aveyron,NaN,Dairy,"December 5, 2021",NaN,"December 6, 2021",Farm,France,...,Yes,Switzerland,"January 13, 2022","44.292650976542205, 2.51639218754828",44.292650976542205,2.51639218754828,44.292651,2.516392,2.801235e+05,5.510843e+06
2,LNM3,Saint_Felicien,Dauphine,NaN,Dairy,"December 5, 2021",NaN,"December 6, 2021",Farm,France,...,Yes,Switzerland,"January 13, 2022","45.48304781253344, 5.174886964350282",45.48304781253344,5.174886964350282,45.483048,5.174887,5.760658e+05,5.697891e+06
3,LNM4,Buchette,Tarn,NaN,Dairy,"December 5, 2021",NaN,"December 6, 2021",Farm,France,...,Yes,Switzerland,"January 13, 2022","43.84129813071417, 2.1575430725683145",43.84129813071417,2.1575430725683145,43.841298,2.157543,2.401766e+05,5.440915e+06
4,LNM5,Rocamadour,Lot,NaN,Dairy,"December 5, 2021",NaN,"December 6, 2021",Farm,France,...,Yes,Switzerland,"January 13, 2022","44.822546764540235, 1.6320714859475425",44.822546764540235,1.6320714859475425,44.822547,1.632071,1.816814e+05,5.593628e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,LNM162,Pla_som_fug,"Talad Ya mo, Muang, Nakhon Ratchasima",NaN,Fish,"March 29, 2022",4,"April 1, 2022",Home,Thailand,...,NaN,Thailand,NaN,"14.971378899712168, 102.08126325325375",14.971378899712168,102.08126325325375,14.971379,102.081263,1.136363e+07,1.685902e+06
162,LNM163,Kai_pla_som,"Talad Ya mo, Muang, Nakhon Ratchasima",Kai_pla_som_evolution,Fish,"March 29, 2022",1,"March 30, 2022",Home,Thailand,...,NaN,Thailand,NaN,"14.971378899712168, 102.08126325325375",14.971378899712168,102.08126325325375,14.971379,102.081263,1.136363e+07,1.685902e+06
163,LNM164,Pla_som_minnow,"Talad Ya mo, Muang, Nakhon Ratchasima",Pla_som_sa_wai_evolution,Fish,"March 29, 2022",1,"March 30, 2022",Home,Thailand,...,NaN,Thailand,NaN,"14.971378899712168, 102.08126325325375",14.971378899712168,102.08126325325375,14.971379,102.081263,1.136363e+07,1.685902e+06
164,LNM165,Pla_som_ta_pien,"Talad Ya mo, Muang, Nakhon Ratchasima",Pla_som_ta_pien_evolution,Fish,"March 29, 2022",1,"March 30, 2022",Home,Thailand,...,NaN,Thailand,NaN,"14.971378899712168, 102.08126325325375",14.971378899712168,102.08126325325375,14.971379,102.081263,1.136363e+07,1.685902e+06


In [7]:
class_options = list(ferm_locations_final.Class.unique())

class_selector = bokeh.models.Select(
    title="class", options=class_options, value="Dairy", width=200,
)

In [8]:
ferm_locations_final['color'] = ["#1f77b3"] * len(ferm_locations_final)

In [9]:
chosen_cds = bokeh.models.ColumnDataSource(
    {
        "x": ferm_locations_final["x"],
        "y": ferm_locations_final["y"],
        "color": ["#1f77b3"] * len(ferm_locations_final),
    }
)

In [10]:
# point available
cds = bokeh.models.ColumnDataSource(ferm_locations_final)

# Make the plot

In [11]:
p = bokeh.plotting.figure(
    frame_height=400,
    frame_width=700,
    x_axis_label="Lat",
    y_axis_label="Long",
    tools =['hover', 'wheel_zoom', 'pan', 'reset'],
    x_axis_type = 'mercator',
    y_axis_type = 'mercator',
    x_range = [ferm_locations_final.x.min()-100000, ferm_locations_final.x.max() + 100000],
    y_range = [ferm_locations_final.y.min()-100000, ferm_locations_final.y.max() + 100000]
    
)
p.add_tile(tile_provider)

circle = p.circle(
    source=chosen_cds, x="x", y="y", fill_color="color", line_color="color",
)

# Define the func

In [12]:
# from https://stackoverflow.com/questions/52260556/bokeh-changing-points-on-map-based-on-slider

js_funs = {
"selector": """
function callback() {
    let selected_class = class_selector.value;
    
    let point_visible = source_visible.data;
    let point_available = source_available.data;
    
    point_visible.x = []
    point_visible.y = []
    point_visible.color = []

    for (let i = 0; i < point_available.x.length; i++) {
        if (point_available.Class[i] == selected_class) {
            point_visible.x.push(point_available.x[i]);
            point_visible.y.push(point_available.y[i]);
            point_visible.color.push('#1f77b3');
            
        }
    }
    console.log(point_visible.x.length);
    console.log(point_visible.y.length);
    console.log(point_visible.color.length);

    source_visible.change.emit();
}
    
"""   
}

In [13]:
# # from https://stackoverflow.com/questions/52260556/bokeh-changing-points-on-map-based-on-slider

# js_funs = {
# "selector": """
# function callback() {
#     let selected_class = class_selector.value;
    
#     console.log(source_available.data);

#     source_visible.data.x = source_available.data.x.filter(d => d.Class == selected_class);
#     source_visible.data.y = source_available.data.y.filter(d => d.Class == selected_class);

#     source_visible.change.emit();
# }
    
# """   
# }

In [14]:
# JavaScript callback
js_code = js_funs["selector"] + "callback()"
callback = bokeh.models.CustomJS(
    args=dict(source_available=cds, source_visible=chosen_cds, class_selector=class_selector), code=js_code,
)
class_selector.js_on_change("value", callback)

In [15]:
%%javascript
console.log("Hello World!")

<IPython.core.display.Javascript object>

In [16]:
layout = bokeh.layouts.row(
    p, bokeh.models.Spacer(width=30), class_selector
)

In [17]:
bokeh.io.show(layout)

In [18]:
bokeh.io.save(layout, "test_map.html")

/opt/miniconda3/envs/basics/lib/python3.9/site-packages/bokeh/io/saving.py:142: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/miniconda3/envs/basics/lib/python3.9/site-packages/bokeh/io/saving.py:154: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/Users/lianamerk/git/fermap/test_map.html'